In [0]:
%pip install nltk

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
# configs = {"fs.azure.account.key.storage001de.blob.core.windows.net": "MA38y0ZA2Ij1L3QXl/F/Y8EGXR1VjTGb896PNbUmPT99uhVLoi0qCEiBEBagBQ4yscqwq9Zxb0vX+AStlr0Z3Q=="}

# # create mount to model
# dbutils.fs.mount(
# 	source = "wasbs://model@storage001de.blob.core.windows.net/",
# 	mount_point = "/mnt/datalake/model",
# 	extra_configs = configs
# )

In [0]:
dbutils.library.restartPython()
# df_movie = df_movie.dropDuplicates(["id"])
# df_movie = df_movie.sort("id")

In [0]:
df_movie = spark.read.table('silver.movie')
df_movie.show(5)

+---+---------+-----------------+----------+--------------------+------+---------+--------------------+--------+--------------------+--------------------+----------+--------+--------+
| id|   imdbId|            title|   release|                plot|rating|voteCount|             tagline|language|           companies|           countries|popularity|  budget| revenue|
+---+---------+-----------------+----------+--------------------+------+---------+--------------------+--------+--------------------+--------------------+----------+--------+--------+
|  1|tt0035423|   Kate & Leopold|2002-02-14|An English Duke f...|   6.4|    87814|If they lived in ...|      en|[{'name': 'Mirama...|[{'iso_3166_1': '...|  7.196356|48000000|76019048|
|  6|tt0053580|        The Alamo|1961-07-14|In 1836, a small ...|   6.8|    17070|The Mission That ...|      en|[{'name': 'United...|[{'iso_3166_1': '...|   4.73816|12000000| 7900000|
| 12|tt0053779|    La Dolce Vita|1960-06-10|A series of stori...|   8.0|    7668

In [0]:
df_movie = spark.read.table('silver.movie')
df_keyword = spark.read.table('silver.keyword')
df_cast = spark.read.table('silver.cast')
df_director = spark.read.table('silver.director')
df_genre = spark.read.table('silver.genre')

In [0]:
df_movie = df_movie.select('id', 'imdbId', 'title', 'plot', 'tagline')
df_movie.show(5)

+---+---------+-----------------+--------------------+--------------------+
| id|   imdbId|            title|                plot|             tagline|
+---+---------+-----------------+--------------------+--------------------+
|  1|tt0035423|   Kate & Leopold|An English Duke f...|If they lived in ...|
|  6|tt0053580|        The Alamo|In 1836, a small ...|The Mission That ...|
| 12|tt0053779|    La Dolce Vita|A series of stori...|The world’s most ...|
| 13|tt0053793|     Elmer Gantry|A fast-talking tr...|If there was a do...|
| 16|tt0053976|The Virgin Spring|In 14th-century S...|Ravished innocenc...|
+---+---------+-----------------+--------------------+--------------------+
only showing top 5 rows



In [0]:
df_keyword = df_keyword.withColumnRenamed("movieId","id")
df_cast = df_cast.withColumnRenamed("movieId","id")
df_director = df_director.withColumnRenamed("movieId","id")
df_genre = df_genre.withColumnRenamed("movieId","id")

In [0]:
df_movie = df_movie.join(df_keyword , on=['id'] , how = 'left')
df_movie = df_movie.join(df_cast , on=['id'] , how = 'left')
df_movie = df_movie.join(df_director , on=['id'] , how = 'left')
df_movie = df_movie.join(df_genre , on=['id'] , how = 'left')

df_movie.show(5)

+---+---------+-----------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+
| id|   imdbId|            title|                plot|             tagline|            keywords|                cast|        director|              genres|
+---+---------+-----------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+
|  1|tt0035423|   Kate & Leopold|An English Duke f...|If they lived in ...|[marriage for mon...|[Andrew Jack, Art...|   James Mangold|[Comedy, Fantasy,...|
|  6|tt0053580|        The Alamo|In 1836, a small ...|The Mission That ...|[texas, assault, ...|[Aissa Wayne, Bil...|      John Wayne|[War, Western, Hi...|
| 12|tt0053779|    La Dolce Vita|A series of stori...|The world’s most ...|[night life, love...|[Alain Cuny, Alai...|Federico Fellini|     [Drama, Comedy]|
| 13|tt0053793|     Elmer Gantry|A fast-talking tr...|If there w

In [0]:
df_movie = df_movie.dropDuplicates(subset=['id', 'imdbId'])
df_movie = df_movie.sort('id')
df_movie.count()

109544

##### Convert into pandas

In [0]:
smd = df_movie.toPandas()
smd.head()

,id,imdbId,title,plot,tagline,keywords,cast,director,genres
0,1,tt0035423,Kate & Leopold,An English Duke from 1876 is inadvertedly drag...,"If they lived in the same century, they'd be p...","[marriage for money, love of one's life, duke,...","[Andrew Jack, Arthur J. Nascarella, Bradley Wh...",James Mangold,"[Comedy, Fantasy, Romance]"
1,2,tt0053273,The Shakedown,"After being released from prison, Augie Corton...",Models were the bait for BLACKMAIL!,"[person in a sack, female police officer, tied...","[Charles Lamb, Dorinda Stevens, Douglas Bradle...",John Lemont,"[Drama, Crime]"
2,3,tt0053459,Eyes Without a Face,A surgeon causes an accident which leaves his ...,Beautiful women were the victims of his fiendi...,"[ulcer, film noir, surgeon, experiment, dove, ...","[Brigitte Juslin, Béatrice Altariba, Charles B...",Georges Franju,"[Horror, Drama]"
3,4,tt0053472,Breathless,A small-time thief steals a car and impulsivel...,None,"[car thief, loss of loved one, hotel room, jou...","[André S. Labarthe, Claude Mansard, François M...",Jean-Luc Godard,"[Drama, Crime]"
4,5,tt0053559,13 Ghosts,A family inherits what proves to be a haunted ...,IT'S FUN TO BE SCARED BY 13 GHOSTS,"[mask, mansion, museum, ghost, revenge, séance...","[Darryl Scott McFadden, David Hoffman, Donald ...",William Castle,"[Mystery, Horror]"


In [0]:
smd = smd.dropna(subset=['title','plot','tagline','keywords', 'cast', 'director', 'genres'])

##### Array -> to List

In [0]:
import numpy as np
for feature in ['cast', 'genres', 'keywords']:
    smd[feature] = smd[feature].apply(lambda x: list(x) if isinstance(x, np.ndarray) else [x])


#### Tagline & plot

In [0]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['plot'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

#### Cast

In [0]:
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [0]:
smd

,id,imdbId,title,plot,tagline,keywords,cast,director,genres,description
0,1,tt0035423,Kate & Leopold,An English Duke from 1876 is inadvertedly drag...,"If they lived in the same century, they'd be p...","[marriage for money, love of one's life, duke,...","[Andrew Jack, Arthur J. Nascarella, Bradley Wh...",James Mangold,"[Comedy, Fantasy, Romance]",An English Duke from 1876 is inadvertedly drag...
1,2,tt0053273,The Shakedown,"After being released from prison, Augie Corton...",Models were the bait for BLACKMAIL!,"[person in a sack, female police officer, tied...","[Charles Lamb, Dorinda Stevens, Douglas Bradle...",John Lemont,"[Drama, Crime]","After being released from prison, Augie Corton..."
2,3,tt0053459,Eyes Without a Face,A surgeon causes an accident which leaves his ...,Beautiful women were the victims of his fiendi...,"[ulcer, film noir, surgeon, experiment, dove, ...","[Brigitte Juslin, Béatrice Altariba, Charles B...",Georges Franju,"[Horror, Drama]",A surgeon causes an accident which leaves his ...
4,5,tt0053559,13 Ghosts,A family inherits what proves to be a haunted ...,IT'S FUN TO BE SCARED BY 13 GHOSTS,"[mask, mansion, museum, ghost, revenge, séance...","[Darryl Scott McFadden, David Hoffman, Donald ...",William Castle,"[Mystery, Horror]",A family inherits what proves to be a haunted ...
5,6,tt0053580,The Alamo,"In 1836, a small band of soldiers sacrifice th...",The Mission That Became a Fortress! The Fortre...,"[texas, assault, mexican army, alamo]","[Aissa Wayne, Bill Daniel, Carlos Arruza]",John Wayne,"[War, Western, History, Drama, Adventure]","In 1836, a small band of soldiers sacrifice th..."
...,...,...,...,...,...,...,...,...,...,...
108386,108389,tt2140411,Solace,Six people from different walks of life search...,Alone is sometimes best,[one word title],"[Anthony R. McClara, Dixie Light, Ric Reitz]",Vandon Gibbs,[Thriller],Six people from different walks of life search...
108392,108395,tt2072277,The Wisdom Tree,"In a meditative sci-fi journey, eastern spirit...",Science meets Art meets Mysticism,"[fine art, classical music, neuroscientist, qu...","[Aditi Kisara, Andrea Kulkarni, Cristina DelVa...",Sunil Shah,"[Mystery, Sci-Fi, Drama]","In a meditative sci-fi journey, eastern spirit..."
108399,108402,tt2085752,Autumn Blood,"High in the mountains, two orphaned children k...",Season of vengeance.,"[nude swimming, rifle, alpine village, social ...","[Annica McCrudden, Elias Doblander, George Lenz]",Markus Blunder,"[Thriller, Drama]","High in the mountains, two orphaned children k..."
108404,108407,tt2191605,Addict,Disillusioned sales manager David Pettigrew is...,Some addictions can't be broken,"[psychological thriller, addict, prostitute]","[Brandon Cooke, Carl Austin, E Tegan O'Brien]",Geoff Harmer,[Drama],Disillusioned sales manager David Pettigrew is...


#### Director

In [0]:
smd['director'] = smd['director'].apply(lambda x: [x,x,x])
smd

,id,imdbId,title,plot,tagline,keywords,cast,director,genres,description
0,1,tt0035423,Kate & Leopold,An English Duke from 1876 is inadvertedly drag...,"If they lived in the same century, they'd be p...","[marriage for money, love of one's life, duke,...","[Andrew Jack, Arthur J. Nascarella, Bradley Wh...","[James Mangold, James Mangold, James Mangold]","[Comedy, Fantasy, Romance]",An English Duke from 1876 is inadvertedly drag...
1,2,tt0053273,The Shakedown,"After being released from prison, Augie Corton...",Models were the bait for BLACKMAIL!,"[person in a sack, female police officer, tied...","[Charles Lamb, Dorinda Stevens, Douglas Bradle...","[John Lemont, John Lemont, John Lemont]","[Drama, Crime]","After being released from prison, Augie Corton..."
2,3,tt0053459,Eyes Without a Face,A surgeon causes an accident which leaves his ...,Beautiful women were the victims of his fiendi...,"[ulcer, film noir, surgeon, experiment, dove, ...","[Brigitte Juslin, Béatrice Altariba, Charles B...","[Georges Franju, Georges Franju, Georges Franju]","[Horror, Drama]",A surgeon causes an accident which leaves his ...
4,5,tt0053559,13 Ghosts,A family inherits what proves to be a haunted ...,IT'S FUN TO BE SCARED BY 13 GHOSTS,"[mask, mansion, museum, ghost, revenge, séance...","[Darryl Scott McFadden, David Hoffman, Donald ...","[William Castle, William Castle, William Castle]","[Mystery, Horror]",A family inherits what proves to be a haunted ...
5,6,tt0053580,The Alamo,"In 1836, a small band of soldiers sacrifice th...",The Mission That Became a Fortress! The Fortre...,"[texas, assault, mexican army, alamo]","[Aissa Wayne, Bill Daniel, Carlos Arruza]","[John Wayne, John Wayne, John Wayne]","[War, Western, History, Drama, Adventure]","In 1836, a small band of soldiers sacrifice th..."
...,...,...,...,...,...,...,...,...,...,...
108386,108389,tt2140411,Solace,Six people from different walks of life search...,Alone is sometimes best,[one word title],"[Anthony R. McClara, Dixie Light, Ric Reitz]","[Vandon Gibbs, Vandon Gibbs, Vandon Gibbs]",[Thriller],Six people from different walks of life search...
108392,108395,tt2072277,The Wisdom Tree,"In a meditative sci-fi journey, eastern spirit...",Science meets Art meets Mysticism,"[fine art, classical music, neuroscientist, qu...","[Aditi Kisara, Andrea Kulkarni, Cristina DelVa...","[Sunil Shah, Sunil Shah, Sunil Shah]","[Mystery, Sci-Fi, Drama]","In a meditative sci-fi journey, eastern spirit..."
108399,108402,tt2085752,Autumn Blood,"High in the mountains, two orphaned children k...",Season of vengeance.,"[nude swimming, rifle, alpine village, social ...","[Annica McCrudden, Elias Doblander, George Lenz]","[Markus Blunder, Markus Blunder, Markus Blunder]","[Thriller, Drama]","High in the mountains, two orphaned children k..."
108404,108407,tt2191605,Addict,Disillusioned sales manager David Pettigrew is...,Some addictions can't be broken,"[psychological thriller, addict, prostitute]","[Brandon Cooke, Carl Austin, E Tegan O'Brien]","[Geoff Harmer, Geoff Harmer, Geoff Harmer]",[Drama],Disillusioned sales manager David Pettigrew is...


#### Keyword

In [0]:
import pandas as pd
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True) #1 row
s.name = 'keyword'

In [0]:
# Đếm số lần xuất hiện của từng từ khóa
s = s.value_counts()

# Chỉ giữ lại các từ khóa xuất hiện nhiều hơn 1 lần
s = s[s > 1]

In [0]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [0]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

In [0]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
# smd.head(5)

In [0]:
# smd.loc[0, 'keywords']

#### Sanitize data


In [0]:
def sanitize(x):
    if isinstance(x, list):
        #Strip spaces and convert to lowercase
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [0]:
for feature in ['cast', 'director', 'genres', 'keywords']:
    smd[feature] = smd[feature].apply(sanitize)

In [0]:
def ensure_list(x):
    if isinstance(x, list):
        return x
    else:
        return [x]
        

In [0]:
for feature in ['cast', 'director', 'genres', 'keywords', 'description']:
    smd[feature] = smd[feature].apply(ensure_list)


In [0]:
smd['soup'] = smd['genres'] + smd['keywords'] +  smd['director'] + smd['cast'] + smd['description']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))


In [0]:
# smd['soup'].head(5)

In [0]:
smd = smd.head(10000)
# smd.head(2500)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# count = CountVectorizer(analyzer='word', stop_words='english')
# count_matrix = count.fit_transform(smd['soup'])

tf = TfidfVectorizer(analyzer='word', stop_words='english')
tfidf_matrix = tf.fit_transform(smd['soup'])

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
# import pickle
# cosine_sim = pickle.load(open('/dbfs/mnt/datalake/model/cosine_sim1.pkl','rb'))
# indices = pickle.load(open('/dbfs/mnt/datalake/model/indices.pkl','rb'))
# titles = pickle.load(open('/dbfs/mnt/datalake/model/titles.pkl','rb'))

# cosine_sim_count = cosine_similarity(count_matrix, count_matrix)
cosine_sim_tfidf = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [0]:
df = smd.reset_index()
titles = df['title']
# tạo một Series với index của Series mới sẽ là cột 'id' của DataFrame df và giá trị index là index của DataFrame df,
indices = pd.Series(df.index, index=df['title'])

In [0]:
indices

title
Kate & Leopold            0
The Shakedown             1
Eyes Without a Face       2
13 Ghosts                 3
The Alamo                 4
                       ... 
Sweetie                9995
The Tall Guy           9996
Tango & Cash           9997
Tap                    9998
Three Fugitives        9999
Length: 10000, dtype: int64

In [0]:
# import pickle
# with open("/dbfs/mnt/datalake/model/cosine_sim1.pkl", "wb") as file:
#     pickle.dump(cosine_sim, file)

In [0]:
# with open("/dbfs/mnt/datalake/model/indices.pkl", "wb") as file:
#     pickle.dump(indices, file)

In [0]:
# with open("/dbfs/mnt/datalake/model/titles.pkl", "wb") as file:
#     pickle.dump(titles, file)

In [0]:
def get_recommendations(title, cosine_sim):
    idx = indices[title] # nhập id trả về index
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [0]:
get_recommendations('Spider-Man', cosine_sim_tfidf).head(10)

3856                 Spider-Man 3
3392                 Spider-Man 2
5641    Oz the Great and Powerful
1646                      Darkman
4899              Drag Me to Hell
2282       The Quick and the Dead
9050                A Simple Plan
4526       The Amazing Spider-Man
913                 The Evil Dead
3029                     The Gift
Name: title, dtype: object

In [0]:
# def get_recommendations_1(id):
#     idx = indices[id] # nhập id trả về index
#     sim_scores = list(enumerate(cosine_sim_1[idx]))
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#     sim_scores = sim_scores[1:10]
#     movie_indices = [i[0] for i in sim_scores]
#     return titles.iloc[movie_indices]

In [0]:
get_recommendations('Spider-Man', cosine_sim_count).head(10)

3856                         Spider-Man 3
3392                         Spider-Man 2
4526               The Amazing Spider-Man
5641            Oz the Great and Powerful
7014    Spider-Man: Into the Spider-Verse
6179               Spider-Man: Homecoming
4899                      Drag Me to Hell
1646                              Darkman
4815              Spider-Man: No Way Home
913                         The Evil Dead
Name: title, dtype: object